## This Notebook is inspired by this Kernel : 

- https://www.kaggle.com/alincijov/ranzcr-resnet200d-seresnet152d-inference 

The wieghts were imported from : 
- https://www.kaggle.com/ammarali32/resnet200d-public for (Resnet200d) 
- https://www.kaggle.com/ammarali32/seresnet152d-cv9615 for (Seresnet152d) 

- https://www.kaggle.com/yasufuminakama/pytorch-image-models (for Timm Libraries )

Thanks for all those persons

## Import the libraries 

In [ ]:
#!pip install timm 
import sys 
sys.path.append('../input/pytorch-images-seresnet')
sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')


In [ ]:
import os 
from tqdm.notebook import tqdm  

import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 

from sklearn import model_selection 
from sklearn import preprocessing 
from sklearn import metrics 
import torch 
from torch.utils.data import Dataset, DataLoader 
from torch.nn.parameter import Parameter 
import torch.nn as nn 
import torchvision
from torchvision import models
import albumentations 
from albumentations import * 
from albumentations.pytorch import ToTensorV2 

import timm 
from torch.cuda.amp import autocast, GradScaler 

from PIL import Image, ImageFile
import cv2

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Path for Models 

## Define Paths for pretrained models 

In [ ]:
Resnet_model_path = '../input/resnet200d-public/resnet200d_320_CV9632.pth'
Seresnet_model_path = '../input/seresnet152d-cv9615/seresnet152d_320_CV96.15.pth'
Resnet50_32_4D_path = '../input/pretrained-resnext50-32x4d/resnext50_32x4d-7cdf4587.pth'
Effnet_B5 = '../input/efficientnet-pytorch/efficientnet-b5-586e6cc6.pth'
TEST_PATH = '../input/ranzcr-clip-catheter-line-classification/test/'

In [ ]:
os.listdir('../input/ranzcr-clip-catheter-line-classification/train_tfrecords//')[:10]

In [ ]:
DIR = '../input/ranzcr-clip-catheter-line-classification'
train_dir = os.path.join(DIR,'train.csv')
test_dir = os.path.join(DIR, 'sample_submission.csv')

# Read files 

In [ ]:
train_df = pd.read_csv(train_dir)
test_df = pd.read_csv(test_dir)
train_df.head()

### Quick EDA

In [ ]:
train_df[(train_df['ETT - Normal'] == 1) & (train_df['NGT - Normal'] == 1) & (train_df['CVC - Normal'] == 1)].sum()

In [ ]:
train_df.isnull().sum()

# Create target columns ****

In [ ]:
target_columns = []
[target_columns.append(x) for x in test_df.columns if x not in ['StudyInstanceUID','PatientID']]
target_columns

## Simple EDA 

### We see that the data is skewed ****

In [ ]:
for c in range(len(target_columns)):
    plt.figure()
    train_df[target_columns[c]].hist()
    plt.title(target_columns[c])

In [ ]:
train_df['PatientID'].unique().shape[0]

In [ ]:
df_annotate = pd.read_csv('../input/ranzcr-clip-catheter-line-classification/train_annotations.csv')

In [ ]:
df_annotate.head()

In [ ]:
df_annotate.label.value_counts()

# Just checking if the first image is on test or on sample_submission csv 

In [ ]:
if ("1.2.826.0.1.3680043.8.498.10003659706701445041816900371598078663" in test_df.StudyInstanceUID.values) : 
    print ("True")
else : 
    print('False')

In [ ]:
output_dir = './'
if not os.path.exists(output_dir): 
    os.makedirs(output_dir)
    

In [ ]:
IMAGE_SIZE = 640 
BATCH_SIZE = 32 

In [ ]:
class TestDataset(Dataset): 
    def __init__(self,df,resize=None,transform=None): 
        self.df = df 
        self.file_name = df['StudyInstanceUID'].values 
        self.transform = transform
        self.resize = resize 
        
    def __len__(self): 
        return len(self.df)
    
    def __getitem__(self,idx): 
        file_name = self.file_name[idx] 
        file_path = f'{TEST_PATH}/{file_name}.jpg' 
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.resize is not None : 
            image.resize = [
                self.resize[1],
                self.resize[0]
            ]
        
        if self.transform is not None: 
            augmented = self.transform(image=image)
            image = augmented['image']
        return image 
        

In [ ]:
def get_transforms():
        return Compose([
            Resize(IMAGE_SIZE, IMAGE_SIZE),
            Normalize(
            mean = [0.485, 0.456, 0.406], 
            std = [0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])

In [ ]:
test_aug = Compose([
        Resize(IMAGE_SIZE,IMAGE_SIZE),
        Normalize (
            mean = [0.485, 0.456, 0.406], 
            std = [0.229, 0.224, 0.225],
        ), 
        ToTensorV2(),
    ])
    


## Resnet Model 

In [ ]:
#m1 = timm.create_model(model_name='resnet200d_320',pretrained=False)
#timm.list_models()

In [ ]:
class Resnet200D(nn.Module) : 
    def __init__(self, model_name='resnet200d_320'):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=False)
        n_features = self.model.fc.in_features
        self.model.global_pool = nn.Identity()
        self.model.fc = nn.Identity()
        self.pooling = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(n_features, 11)

    def forward(self, x):
        bs = x.size(0)
        features = self.model(x)
        pooled_features = self.pooling(features).view(bs, -1)
        output = self.fc(pooled_features)
        return output

In [ ]:
#m2 = timm.create_model('seresnet152d')
#m2

In [ ]:
class SeResnet152D(nn.Module): 
    def __init__(self, model_name='seresnet152d_320'):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=False)
        n_features = self.model.fc.in_features
        self.model.global_pool = nn.Identity()
        self.model.fc = nn.Identity()
        self.pooling = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(n_features, 11)

    def forward(self, x):
        bs = x.size(0)
        features = self.model(x)
        pooled_features = self.pooling(features).view(bs, -1)
        output = self.fc(pooled_features)
        return output
        

In [ ]:
#m3 = timm.create_model('resnext50_32x4d')
#m3

In [ ]:
class Resnet5032(nn.Module): 
    def __init__(self,model_name = 'resnext50_32x4d'): 
        super().__init__()
        self.model = timm.create_model(model_name,pretrained=False)
        n_features = self.model.fc.in_features
        self.model.global_pool = nn.Identity()
        self.model.fc = nn.Identity()
        self.pooling = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(n_features,11)
    def forward(self,x): 
        bs = x.size(0)
        features = self.model(x)
        pooled_features = self.pooling(features).view(bs,-1)
        output = self.fc(pooled_features)
        return output 

In [ ]:
class EffB5(nn.Module): 
    def __init__(self,model_name = 'efficientnet_b5'): 
        super().__init__()
        self.model = timm.create_model(model_name,pretrained=False)
        n_features = self.model.classifier.in_features
        self.model.global_pool = nn.Identity()
        self.model.classifier = nn.Identity()
        self.pooling = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(n_features,11)
    def forward(self,x): 
        bs = x.size(0)
        features = self.model(x)
        pooled_features = self.pooling(features).view(bs,-1)
        output = self.fc(pooled_features)
        return output 

In [ ]:
#EfficientNet = EffB5()
#EfficientNet.load_state_dict(torch.load(Effnet_B5))

# Load Pretrained models weights 

In [ ]:
seresnet_model = SeResnet152D()
seresnet_model.load_state_dict(torch.load(Seresnet_model_path)['model'])
seresnet_model.eval()
seresnet_model.to(device)

In [ ]:
resnet_model = Resnet200D()
resnet_model.load_state_dict(torch.load(Resnet_model_path)['model'])
resnet_model.eval() 
resnet_model.to(device)


In [ ]:

#resnext5032.load_state_dict(torch.load(Resnet50_32_4D_path))
#weightss= torch.load(Resnet50_32_4D_path)
#resnext5032 = Resnet5032() 
#resnext5032.load_state_dict(weightss)

# Merge Models into list 

In [ ]:
models = [resnet_model,seresnet_model]

# Inference function 

In [ ]:
def inference(models, test_loader, device): 
    probs = []
    for i, (images) in tqdm(enumerate(test_loader),total= len(test_loader)) : 
        images = images.to(device)
        avg_preds = []
        for model in models : 
            with torch.no_grad():
                y_preds1 = model(images)
                y_preds2 = model(images.flip(-1))
            y_preds = (y_preds1.sigmoid().to('cpu').numpy() + y_preds2.sigmoid().to('cpu').numpy()) / 2 
            avg_preds.append(y_preds)
        avg_preds = np.mean(avg_preds,axis=0)
        probs.append(avg_preds)
    probs = np.concatenate(probs)
    return probs
            

In [ ]:
resnet_model

In [ ]:
test_dataset = TestDataset(test_df, transform=test_aug)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, 
                         num_workers=4 , pin_memory=True)
model200D = []
model200D.append(resnet_model)
model152D = []
model152D.append(seresnet_model)
prediction200D = inference(model200D, test_loader, device)
prediction152D = inference(model152D, test_loader, device)
predictions = (2 * prediction200D + prediction152D) / 3.0
#predictions200d = inference(models200D, test_loader, device)
#predictions152d = inference(models152D, test_loader, device)
#predictions = (2 * predictions200d + predictions152d) / 3.0

In [ ]:
target_cols = test_df.iloc[:, 1:12].columns.tolist()
test_df[target_cols] = predictions
test_df[['StudyInstanceUID'] + target_cols].to_csv('submission.csv', index=False)
test_df.head()